# Question Answering 

Part of the output process of the retrieval augmented generation workflow

- Multiple relevant documents have been retrieved from the vector store 
- Potentially compress the relevant splits to fit into the LLM context
- Send the information along with our question to an LLM to select and format an answer

In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [3]:
# Load in our db that was persisted from before
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [4]:
print(vectordb._collection.count())

209


In [5]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [8]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

## RetrievalQA Chain 

In [9]:
from langchain.chains import RetrievalQA

In [10]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [13]:
result = qa_chain.invoke({"query": question})

In [14]:
result["result"]

'The major topics for this class seem to include machine learning, statistics, and algebra. Additionally, there will be discussions on extensions of the material covered in the main lectures.'

## Prompt

In [15]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [16]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [17]:
question = "Is probability a class topic?"

In [19]:
result = qa_chain.invoke({"query": question})

In [20]:
result["result"]

'Yes, probability is a class topic as the instructor assumes familiarity with basic probability and statistics. Thanks for asking!'

In [21]:
# We can check where its getting this data from
result["source_documents"][0]

Document(page_content="of this class will not be very program ming intensive, although we will do some \nprogramming, mostly in either MATLAB or Octa ve. I'll say a bit more about that later.  \nI also assume familiarity with basic proba bility and statistics. So most undergraduate \nstatistics class, like Stat 116 taught here at Stanford, will be more than enough. I'm gonna \nassume all of you know what ra ndom variables are, that all of you know what expectation \nis, what a variance or a random variable is. And in case of some of you, it's been a while \nsince you've seen some of this material. At some of the discussion sections, we'll actually \ngo over some of the prerequisites, sort of as  a refresher course under prerequisite class. \nI'll say a bit more about that later as well.  \nLastly, I also assume familiarity with basi c linear algebra. And again, most undergraduate \nlinear algebra courses are more than enough. So if you've taken courses like Math 51, \n103, Math 113 or 

## RetrievalQA chain types

Map_reduce technique, each of the individual documents are first sent to the LLM by itself to get an original answer and then those answers are composed into a final answer with a final call to the LLM

In [27]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type="map_reduce"
)

In [28]:
result = qa_chain_mr.invoke({"query": question})

In [29]:
result["result"]

'Yes, probability is a class topic in the document.'

In [31]:
result["source_documents"][0]

Document(page_content="of this class will not be very program ming intensive, although we will do some \nprogramming, mostly in either MATLAB or Octa ve. I'll say a bit more about that later.  \nI also assume familiarity with basic proba bility and statistics. So most undergraduate \nstatistics class, like Stat 116 taught here at Stanford, will be more than enough. I'm gonna \nassume all of you know what ra ndom variables are, that all of you know what expectation \nis, what a variance or a random variable is. And in case of some of you, it's been a while \nsince you've seen some of this material. At some of the discussion sections, we'll actually \ngo over some of the prerequisites, sort of as  a refresher course under prerequisite class. \nI'll say a bit more about that later as well.  \nLastly, I also assume familiarity with basi c linear algebra. And again, most undergraduate \nlinear algebra courses are more than enough. So if you've taken courses like Math 51, \n103, Math 113 or 

In [ ]:
#! pip install langsmith

In [44]:
# Load environment keys to trace the chain in LangSmith
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
tracing_v2 = os.environ["LANGCHAIN_TRACING_V2"]
langchain_api_key = os.environ["LANGCHAIN_API_KEY"]
langchain_project = os.environ["LANGCHAIN_PROJECT"]
langchain_endpoint = os.environ["LANGCHAIN_ENDPOINT"]

In [45]:

qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain_mr.invoke({"query": question})
result["result"]

'Yes, probability is a class topic mentioned in the text.'

In [46]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr.invoke({"query": question})
result["result"]

'Based on the additional context provided, it is clear that probability will indeed be a class topic. The instructor mentions using probability concepts in the discussion sections to provide a refresher on statistics and algebra, as well as to cover extensions for the material taught in the main lectures on machine learning. This further confirms that probability will be a key topic in the course.'

## RetrievalQA limitations

QA fails to preserve conversational history.

In [47]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [48]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

'Yes, probability is a class topic mentioned in the context provided. The instructor assumes familiarity with basic probability and statistics for the class.'

In [49]:
# As you can see this follow-up question is not quite what we are looking for and is not related to the original question. Memory will need to be added to the chain to keep track of the original question.
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]

'The prerequisites of basic knowledge of computer science, probability and statistics, and linear algebra are needed for the machine learning class because they form the foundational understanding required to grasp the concepts and algorithms taught in the course. Understanding computer science principles helps in programming and implementing machine learning algorithms. Knowledge of probability and statistics is essential for understanding the mathematical foundations of machine learning algorithms. Familiarity with linear algebra is necessary for understanding the mathematical operations involved in machine learning, such as matrix multiplication and eigenvectors. These prerequisites ensure that students have the necessary background knowledge to effectively engage with the material covered in the class.'